# Automatic differentiation with torch.autograd

신경망을 학습시킬 때, 가장 자주 사용하는 알고리즘은 역전파이다. Back propagation  
이 알고리즘에서, 파라미터들(model weights)은 loss function의 기울기에 따라 주어진 파라미터에 적응된다.

이러한 기울기들을 계산하기 위해서,  
PyTorch는 `torch.autograd`라는 built-in 미분 엔진을 가지고 있다.  
오토그라드는 어떠한 계산그래프의 기울기의 자동 계산도 지원할 수 있다.

제일 단순한 1-layer 신경망과 입력 `x`와 파라미터들 `w` 와 `b`, 그리고 몇몇 loss function을 고려하면,  
PyTorch에서는 다음과 같은 예시로 정의될 수 있다.


In [1]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph

위 코드는 아래 그림을 정의한다 (웹 페이지에 그림 있음 ㅎㅎ)  
이 네트워크에서, `w`와 `b`는 파라미터들이다, 우리가 최적화 해야 할.  
그러므로, 우리는 loss function의 기울기를 계산할 필요가 있다.  
이를 수행하기 위해, 우리는 해당 tensors에 `requires_grad` 프로퍼티를 설정했다.

나중에 설정하려면 `x.requires_grad_(True)` 이런식으로 설정하면 된다.

클래스 객체인 `Function`을 보자.  
이 객체는 `forward` 방향에서 함수가 어떻게 계산하는지를 알고 있다,  
또한 `backward propagation`동안 그것의 도함수를 어떻게 계산하는 지를 알고 있다.  
`backward propagation function`에 대한 참조는 `grad_fn`이란 텐서 프로퍼티에 저장되어 있다.  

In [7]:
print('Gradient function for z = ', z.grad_fn)
print('Gradient function for loss = ', loss.grad_fn)

Gradient function for z =  <AddBackward0 object at 0x7fa091ab03a0>
Gradient function for loss =  <BinaryCrossEntropyWithLogitsBackward object at 0x7fa091ab0f70>


## Computing Gradients

신경망에서 파라미터들의 weights를 최적화 하기 위해서,  
우리는 우리의 loss function의 도함수를 계산할 필요가 있다.  
`x`와 `y`는 고정값이고, `w`와 `b`의 도함수를 계산한다.  
이를 계산하기 위해 `loss.backward()` 를 호출한다,  
그리고 `w.grad`와 `b.grad`에 저장된 값을 확인한다.

In [8]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2415, 0.1202, 0.3299],
        [0.2415, 0.1202, 0.3299],
        [0.2415, 0.1202, 0.3299],
        [0.2415, 0.1202, 0.3299],
        [0.2415, 0.1202, 0.3299]])
tensor([0.2415, 0.1202, 0.3299])


### Note
`requries_grad=True`로 `grad` 프로퍼티가 설정된 노드에만 기울기 값을 갖는다.  

주어진 그래프에 대해서 `backward`는 한번한 호출할 수 있다.
하나의 그래프에 여러번 호출하려면 `retain_graph=True`를 설정해야한다.

## Disabling Gradient Tracking

기본적으로, `requires_grad=True`를 설정한 모든 텐서들은 그들의 계산 기록과 지원되는 기울기 계산을 추적하고 있는다.  
그러나, 필요하지 않은 경우에는  
(예를 들어 우리가 이미 학습된 모델이 있고, 몇몇 input data에 적용만 해보고 싶은 경우)  
우리는 단지 해당 신경망에서 `forward`를 하고 싶을 뿐이다.
그렇다면 `torch.no_grad()` 블록으로 감싸진 계산 코드를 사용하면 추적을 멈출 수 있다.

In [10]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


다른 방법은 텐서에서 `detach()` 메소드를 수행하면 된다

In [11]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


### 기울기 추적을 멈춰야하는 이유

- 너의 신경망에서 몇몇 파라미터는 frozen params로 지정하기 위해.
  - 이것은 finetuning에서 매우 잦은 시나리오이다.
- 오직 `forward`만 수행하려고 할 때 계산 속도를 높이기 위함.

## More on Computational Graphs

개념적으로,  
autograd는 `Function` 객체들을 구성하는 DAG안의  
모든 텐서들과 실행된 명령들의 기록을 저장한다.  

DAG안에서, leaves는 input tensors이고, roots는 output tensors이다.  
이렇게 추적함으로써 chain rule을 사용해서 기울기들을 자동적으로 계산할 수 있다.

`forward` 단계에서, autograd는 동시에 2가지를 수행한다.

- 결과 tensor를 계산하기 위해 요청된 명령을 실행한다
- DAG 안의 명령의 gradient function을 유지한다.

`backward` 단계는 DAG root에서 `.backward()`를 호출하면 실행된다.  
`autograd`는

- `grad_fn` 마다 기울기를 계산한다
- 각각 텐서들의 `.grad` 속성에 그들을 모은다.
- chain rule을 사용해서, leaf 텐서들에 전파한다.

### Note
PyTorch에서 DAG들은 동적이다.  
중요하게 기억할 점은 그래프들은 scratch(빈 상태, 처음 상태)부터 재 생성된다.  
매 `.backward()` 호출 이후, autograd는 새로운 그래프를 생성하기를 시작한다.  
이는 너의 모델 안에서 control flow statements(if나 for 같은 것들)을 사용할 수 있게 한다.  
너는 필요하다면 매 iteration에서 shape나 size, operations를 바꿀 수 있다.

## Optional Reading: Tensor Gradients and Jacobian Products

모든 경우, 우리는 scalar loss function을 갖는다,  
그리고 몇몇 파라미터에 관해서는 기울기를 계산할 필요가 있다.  
그러나, output function이 임의의 텐서인 경우들이 있다. -> scalar가 아니라 텐서.  

이러한 경우, 파이토치는 Jacobian product를 계산하는 것을 허용한다,  진짜 기울기가 아닌.

-> 깊은 내용은 따로 정리

In [13]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
